In [161]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [162]:
annal_report=pd.read_csv('train/annual_report_info.csv')
entprise_info=pd.read_csv('train/entprise_info.csv')
entprise_evaluate=pd.read_csv("entprise_evaluate.csv")

In [163]:
del annal_report['STATE']#这玩意没啥用 都是0 删掉删掉
del annal_report['MEMNUM']#也没用
del annal_report['FARNUM']#一样没用
del annal_report['ANNREDMEMNUM']
del annal_report['ANNNEWMEMNUM']

In [164]:
annal_report.describe()

,ANCHEYEAR,FUNDAM,EMPNUM,EMPNUMSIGN,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
count,22550.000000,5702.000000,22535.000000,16833.000000,20041.000000,20041.000000,20041.000000,20041.000000,20041.000000,20041.000000,20041.000000,20041.000000,22517.000000,16489.000000,13507.000000,22530.000000
mean,2016.805144,128.334488,11.789927,1.835799,0.894017,0.059029,0.017115,0.316801,1.687690,0.104436,0.037972,0.808243,1.963272,1.941173,1.946028,2.709632
std,1.084357,6784.691190,142.248117,0.370469,6.349607,0.955736,0.147353,2.516092,10.314762,1.252551,0.551450,6.136716,0.188097,0.235308,0.225971,0.560448
min,2015.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,2016.000000,3.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,3.000000
50%,2017.000000,7.150000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,3.000000
75%,2018.000000,10.000000,5.000000,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,3.000000
max,2018.000000,500000.000000,12045.000000,2.000000,660.000000,120.000000,5.000000,205.000000,660.000000,80.000000,42.000000,340.000000,2.000000,2.000000,2.000000,3.000000


In [165]:
annal_report[annal_report['EMPNUMSIGN']==2]

,id,ANCHEYEAR,FUNDAM,EMPNUM,EMPNUMSIGN,BUSSTNAME,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2017.0,NaN,4.0,2.0,开业,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2018.0,NaN,3.0,2.0,开业,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
7,e9f7b28ec10e0470287f274dd5a327519e74d2eb9506faad,2016.0,NaN,8.0,2.0,开业,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0
8,e9f7b28ec10e0470287f274dd5a327519e74d2eb9506faad,2017.0,NaN,8.0,2.0,开业,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0
9,e9f7b28ec10e0470287f274dd5a327519e74d2eb9506faad,2018.0,NaN,1.0,2.0,开业,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22545,f000950527a6feb6bde38216d7cbbf32e66d3a3a96d4dbda,2015.0,NaN,3.0,2.0,开业,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,NaN,2.0
22546,516ab81418ed215dcbbf0614a7b929e691f8eed153d7bb31,2015.0,NaN,8.0,2.0,开业,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
22547,516ab81418ed215dcbbf0614a7b929e691f8eed153d7bb31,2016.0,NaN,4.0,2.0,歇业,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
22548,516ab81418ed215dcbbf0614a7b929e691f8eed153d7bb31,2017.0,NaN,4.0,2.0,歇业,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0


In [166]:
for col in annal_report.columns:
    print(col,annal_report[col].isnull().sum())

id 0
ANCHEYEAR 0
FUNDAM 16848
EMPNUM 15
EMPNUMSIGN 5717
BUSSTNAME 4870
COLGRANUM 2509
RETSOLNUM 2509
DISPERNUM 2509
UNENUM 2509
COLEMPLNUM 2509
RETEMPLNUM 2509
DISEMPLNUM 2509
UNEEMPLNUM 2509
WEBSITSIGN 33
FORINVESTSIGN 6061
STOCKTRANSIGN 9043
PUBSTATE 20


FUNDAM 16848   //用中位数填充
EMPNUM 15       //用中位数填充
EMPNUMSIGN 5717  //用-1填充
BUSSTNAME 4870    //换成 1 开业 0 歇业 -1 nan
COLGRANUM 2509      //用中位数填充
RETSOLNUM 2509      //用中位数填充
DISPERNUM 2509      //用中位数填充
UNENUM 2509         //中位数
COLEMPLNUM 2509     //中位数 
RETEMPLNUM 2509     //中位数
DISEMPLNUM 2509     //中位数
UNEEMPLNUM 2509     //中位数
WEBSITSIGN 33       //中位数 or -1填充
FORINVESTSIGN 6061      //-1填充
STOCKTRANSIGN 9043      //-1填充
PUBSTATE 20         //中位数or -1 

In [167]:
annal_report.head()

,id,ANCHEYEAR,FUNDAM,EMPNUM,EMPNUMSIGN,BUSSTNAME,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2017.0,5.0,10.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2018.0,2.0,2.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2017.0,NaN,4.0,2.0,开业,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2018.0,NaN,3.0,2.0,开业,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
4,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2017.0,5.0,10.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0


In [168]:
annal_report.dtypes

id                object
ANCHEYEAR        float64
FUNDAM           float64
EMPNUM           float64
EMPNUMSIGN       float64
BUSSTNAME         object
COLGRANUM        float64
RETSOLNUM        float64
DISPERNUM        float64
UNENUM           float64
COLEMPLNUM       float64
RETEMPLNUM       float64
DISEMPLNUM       float64
UNEEMPLNUM       float64
WEBSITSIGN       float64
FORINVESTSIGN    float64
STOCKTRANSIGN    float64
PUBSTATE         float64
dtype: object

In [169]:
entprise_evaluate.columns=['id','label']
for column in entprise_evaluate.columns:
    print(column,entprise_evaluate[column].isnull().sum())

id 0
label 10000


In [170]:
labels=pd.concat([entprise_info,entprise_evaluate])
labels.head()

,id,label
0,59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7,1.0
1,da8691b210adb3f6be8064e006f220070565db287275ad38,0.0
2,82750f1b9d122350918121f97c99bf96e11aa24ee91504a9,0.0
3,f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec,0.0
4,f1c1045b13d1832927e3743e49d2917f2d98424f0849a373,0.0


In [171]:
labels.isnull().sum()

id           0
label    10000
dtype: int64

In [172]:
annal=pd.merge(annal_report,labels,on='id',how='left')

In [173]:
for col in ['FUNDAM','EMPNUM','COLGRANUM','RETSOLNUM','DISPERNUM','UNENUM','COLEMPLNUM','RETEMPLNUM','DISEMPLNUM','UNEEMPLNUM','WEBSITSIGN','PUBSTATE']:
    annal[col]=annal[col].fillna(annal[col].median())

for col in ['EMPNUMSIGN','FORINVESTSIGN','STOCKTRANSIGN']:
    annal[col]=annal[col].fillna(-1).astype('int')

lb=LabelEncoder()
annal['BUSSTNAME']=lb.fit_transform(annal['BUSSTNAME'].astype(str))

In [174]:
for col in annal.columns:
    print(col,annal[col].isnull().sum())

id 0
ANCHEYEAR 0
FUNDAM 0
EMPNUM 0
EMPNUMSIGN 0
BUSSTNAME 0
COLGRANUM 0
RETSOLNUM 0
DISPERNUM 0
UNENUM 0
COLEMPLNUM 0
RETEMPLNUM 0
DISEMPLNUM 0
UNEEMPLNUM 0
WEBSITSIGN 0
FORINVESTSIGN 0
STOCKTRANSIGN 0
PUBSTATE 0
label 15730


In [175]:
train=annal[annal.label.notna()]
test=annal[annal.label.isna()]
print(train.shape,test.shape)

(6820, 19) (15730, 19)


In [176]:
model=lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           tree_learner='serial',
                           num_leaves=100,#调这个参数没用
                           max_depth=12,#有点点用
                           learning_rate=0.015,#别动
                           n_estimators=100000,#没用
                           subsample=0.8,
                           feature_fraction=0.8,
                           reg_alpha=0.3,
                           reg_lambda=0.5,
                           random_state=2020,
                           is_unbalance=True)